In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
! export TF_CPP_MIN_LOG_LEVEL=2

## Hardware check

In [3]:
# gpu check
!nvidia-smi

Wed Dec 13 11:12:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3B:00.0 Off |                    0 |
| N/A   34C    P0    39W / 300W |      9MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:B2:00.0 Off |                    0 |
| N/A   

In [4]:
# check number of cores
import multiprocessing

cores = multiprocessing.cpu_count() 
cores

80

## Environment Prep

In [5]:
cd /pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis

/pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis


In [6]:
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Lambda, GlobalAveragePooling1D
# from tensorflow.keras.models import Model
from tensorflow.keras import models
import pickle
import numpy as np
from tqdm import tqdm
tqdm.pandas()
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback, EarlyStopping
import pandas as pd
import json
from torch.utils.data import Dataset
from transformers import AutoTokenizer, pipeline, AutoModel
import resources.smart_cond as sc
# from google.colab import files
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer

## Version Check

In [7]:
import tensorflow as tf
print(tf. __version__)

pickle.format_version

2.12.0


'4.0'

## Load Data

In [8]:
data_path = 'Data/sepsis_removed_0.pkl'
pkl = pickle.load(open(data_path, 'rb'))
data = pkl[0]
oc = pkl[1]
train_ind = pkl[2]
valid_ind = pkl[3]
test_ind = pkl[4]
del pkl

In [9]:
data

,ts_ind,hour,variable,value,TABLE,mean,std
0,10223,467.816667,Text,Admission Date: [**2119-5-4**] D...,noteevents,1.000000,1.000000
1,18407,28.016667,Text,Admission Date: [**2112-12-8**] ...,noteevents,1.000000,1.000000
2,40300,155.166667,Text,Admission Date: [**2194-7-18**] ...,noteevents,1.000000,1.000000
3,23747,52.383333,Text,Admission Date: [**2194-1-7**] D...,noteevents,1.000000,1.000000
4,2357,73.133333,Text,Admission Date: [**2186-6-7**] Discharge ...,noteevents,1.000000,1.000000
...,...,...,...,...,...,...,...
82886223,57281,20.400000,MBP,0.195381,chart,78.552377,17.645628
82886224,57281,20.400000,O2 Saturation,-0.678068,chart,96.820961,4.160290
82886225,57281,20.400000,RR,0.179866,chart,26.278501,15.130729
82886226,57281,20.400000,SBP,-0.404061,chart,120.239648,25.341836


In [10]:
data.loc[data['variable'] == 'Text', 'value'] = 1
data

,ts_ind,hour,variable,value,TABLE,mean,std
0,10223,467.816667,Text,1,noteevents,1.000000,1.000000
1,18407,28.016667,Text,1,noteevents,1.000000,1.000000
2,40300,155.166667,Text,1,noteevents,1.000000,1.000000
3,23747,52.383333,Text,1,noteevents,1.000000,1.000000
4,2357,73.133333,Text,1,noteevents,1.000000,1.000000
...,...,...,...,...,...,...,...
82886223,57281,20.400000,MBP,0.195381,chart,78.552377,17.645628
82886224,57281,20.400000,O2 Saturation,-0.678068,chart,96.820961,4.160290
82886225,57281,20.400000,RR,0.179866,chart,26.278501,15.130729
82886226,57281,20.400000,SBP,-0.404061,chart,120.239648,25.341836


In [11]:
pred_window = 2 # hours
obs_windows = range(20, 124, 4)
# Remove test patients.
data = data.merge(oc[['ts_ind', 'SUBJECT_ID']], on='ts_ind', how='left')
test_sub = oc.loc[oc.ts_ind.isin(test_ind)].SUBJECT_ID.unique()
data = data.loc[~data.SUBJECT_ID.isin(test_sub)]
oc = oc.loc[~oc.SUBJECT_ID.isin(test_sub)]
data.drop(columns=['SUBJECT_ID', 'TABLE'], inplace=True)
# Fix age.
data.loc[(data.variable=='Age')&(data.value>200), 'value'] = 91.4
# data[data.variable=='Age'][data.value>200]['value'] = 91.4

In [12]:
# Get static data with mean fill and missingness indicator.
static_varis = ['Age', 'Gender']
ii = data.variable.isin(static_varis)
static_data = data.loc[ii]
data = data.loc[~ii]
def inv_list(l, start=0):
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d
static_var_to_ind = inv_list(static_varis)
D = len(static_varis)
N = data.ts_ind.max()+1
demo = np.zeros((N, D))
for row in tqdm(static_data.itertuples()):
    demo[row.ts_ind, static_var_to_ind[row.variable]] = row.value
# Normalize static data.
means = demo.mean(axis=0, keepdims=True)
stds = demo.std(axis=0, keepdims=True)
stds = (stds==0)*1 + (stds!=0)*stds
demo = (demo-means)/stds
# Get variable indices.
varis = sorted(list(set(data.variable)))
V = len(varis)
var_to_ind = inv_list(varis, start=1)
data['vind'] = data.variable.map(var_to_ind)
data = data[['ts_ind', 'vind', 'hour', 'value']].sort_values(by=['ts_ind', 'vind', 'hour'])
# Find max_len.
fore_max_len = 880
# Get forecast inputs and outputs.
fore_times_ip = []
fore_values_ip = []
fore_varis_ip = []
fore_op = []
fore_inds = []
def f(x):
    mask = [0 for i in range(V)]
    values = [0 for i in range(V)]
    for vv in x:
        v = int(vv[0])-1
        mask[v] = 1
        values[v] = vv[1]
    return values+mask
def pad(x):
    return x+[0]*(fore_max_len-len(x))
for w in tqdm(obs_windows):
    pred_data = data.loc[(data.hour>=w)&(data.hour<=w+pred_window)]
    pred_data = pred_data.groupby(['ts_ind', 'vind']).agg({'value':'first'}).reset_index()
    pred_data['vind_value'] = pred_data[['vind', 'value']].values.tolist()
    pred_data = pred_data.groupby('ts_ind').agg({'vind_value':list}).reset_index()
    pred_data['vind_value'] = pred_data['vind_value'].apply(f)    
    obs_data = data.loc[(data.hour<w)&(data.hour>=w-24)]
    obs_data = obs_data.loc[obs_data.ts_ind.isin(pred_data.ts_ind)]
    obs_data = obs_data.groupby('ts_ind').head(fore_max_len)
    obs_data = obs_data.groupby('ts_ind').agg({'vind':list, 'hour':list, 'value':list}).reset_index()
    obs_data = obs_data.merge(pred_data, on='ts_ind')
    for col in ['vind', 'hour', 'value']:
        obs_data[col] = obs_data[col].apply(pad)
    fore_op.append(np.array(list(obs_data.vind_value)))
    fore_inds.append(np.array(list(obs_data.ts_ind)))
    fore_times_ip.append(np.array(list(obs_data.hour)))
    fore_values_ip.append(np.array(list(obs_data.value)))
    fore_varis_ip.append(np.array(list(obs_data.vind)))
del data
fore_times_ip = np.concatenate(fore_times_ip, axis=0)
fore_values_ip = np.concatenate(fore_values_ip, axis=0)
fore_varis_ip = np.concatenate(fore_varis_ip, axis=0)
fore_op = np.concatenate(fore_op, axis=0)
fore_inds = np.concatenate(fore_inds, axis=0)
fore_demo = demo[fore_inds]
# Get train and valid ts_ind for forecast task.
# train_sub = oc.loc[oc.ts_ind.isin(train_ind)].SUBJECT_ID.unique()
valid_sub = oc.loc[oc.ts_ind.isin(valid_ind)].SUBJECT_ID.unique()
rem_sub = oc.loc[~oc.SUBJECT_ID.isin(np.concatenate((train_ind, valid_ind)))].SUBJECT_ID.unique()
bp = int(0.8*len(rem_sub))
# train_sub = np.concatenate((train_sub, rem_sub[:bp]))
valid_sub = np.concatenate((valid_sub, rem_sub[bp:]))
# train_ind = oc.loc[oc.SUBJECT_ID.isin(train_sub)].ts_ind.unique() # Add remaining ts_ind s of train subjects.
valid_ind = oc.loc[oc.SUBJECT_ID.isin(valid_sub)].ts_ind.unique() # Add remaining ts_ind s of train subjects.
# Generate 3 sets of inputs and outputs.
# train_ind = np.argwhere(np.in1d(fore_inds, train_ind)).flatten()
valid_ind = np.argwhere(np.in1d(fore_inds, valid_ind)).flatten()
# fore_train_ip = [ip[train_ind] for ip in [fore_demo, fore_times_ip, fore_values_ip, fore_varis_ip]]
fore_valid_ip = [ip[valid_ind] for ip in [fore_demo, fore_times_ip, fore_values_ip, fore_varis_ip]]
del fore_times_ip, fore_values_ip, fore_varis_ip, demo, fore_demo
# fore_train_op = fore_op[train_ind]
fore_valid_op = fore_op[valid_ind]
del fore_op

91626it [00:00, 784217.39it/s]
100%|██████████| 26/26 [04:54<00:00, 11.34s/it]


In [13]:
train_path = 'Exp_GloVe/texts_ip/forecasting_train_ip.txt'

train_text_ip = []

with open(train_path) as file:
    for line in tqdm(file):
        train_text_ip.append(line.rstrip())

# train_text_ip = np.array(train_text_ip)

valid_path = 'Exp_GloVe/texts_ip/forecasting_valid_ip.txt'

valid_text_ip = []

with open(valid_path) as file:
    for line in tqdm(file):
        valid_text_ip.append(line.rstrip())
        
# valid_text_ip = np.array(valid_text_ip)

449482it [00:03, 113950.89it/s]
136823it [00:01, 104369.69it/s]


In [14]:
# prepare tokenizer
t = Tokenizer()
train_val_ip = train_text_ip+valid_text_ip
t.fit_on_texts(train_val_ip)
vocab_size = len(t.word_index) + 1
print(f'vocabulary size: {vocab_size}')
del train_val_ip

# encode
encoded_train = t.texts_to_sequences(train_text_ip)
encoded_valid = t.texts_to_sequences(valid_text_ip)

# pad to max_len
max_length = 24135
padded_train = pad_sequences(encoded_train, maxlen=max_length, padding='post')
padded_valid = pad_sequences(encoded_valid, maxlen=max_length, padding='post')

vocabulary size: 297321


In [15]:
# fore_train_ip.append(padded_train)
fore_valid_ip.append(padded_valid)

del train_text_ip, valid_text_ip

In [16]:
from numpy import array, asarray, zeros

# GloVe
glove_path = 'resources/glove.840B.300d.txt'

embedding_model = {}
f = open('resources/glove.840B.300d.txt', 'r', encoding="utf8")
for line in tqdm(f):
    values = line.split()
    word = ''.join(values[:-300])
    coefs = np.asarray(values[-300:], dtype='float32')
    embedding_model[word] = coefs
f.close()

2196017it [01:31, 24070.65it/s]


In [17]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 300))
for word, i in tqdm(t.word_index.items()):
    embedding_vector = embedding_model.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 297320/297320 [00:01<00:00, 287520.94it/s]


In [18]:
def get_res(y_true, y_pred):
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
    pr_auc = auc(recall, precision)
    minrp = np.minimum(precision, recall).max()
    roc_auc = roc_auc_score(y_true, y_pred)
    return [roc_auc, pr_auc, minrp]

# ######################################################################################################## 
# ######################################################################################################## 
# class_weights = compute_class_weight(class_weight='balanced', classes=[0,1], y=train_op)
# def mortality_loss(y_true, y_pred):
#     sample_weights = (1-y_true)*class_weights[0] + y_true*class_weights[1]
#     bce = K.binary_crossentropy(y_true, y_pred)
#     return K.mean(sample_weights*bce, axis=-1)
# ######################################################################################################## 
# ######################################################################################################## 

# var_weights = np.sum(fore_train_op[:, V:], axis=0)
# var_weights[var_weights==0] = var_weights.max()
# var_weights = var_weights.max()/var_weights
# var_weights = var_weights.reshape((1, V))
def forecast_loss(y_true, y_pred):
    return K.sum(y_true[:,V:]*(y_true[:,:V]-y_pred)**2, axis=-1)

def get_min_loss(weight):
    def min_loss(y_true, y_pred):
        return weight*y_pred
    return min_loss

class CustomCallback(Callback):
    def __init__(self, validation_data, batch_size):
        self.val_x, self.val_y = validation_data
        self.batch_size = batch_size
        super(Callback, self).__init__()

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.val_x, verbose=0, batch_size=self.batch_size)
        if type(y_pred)==type([]):
            y_pred = y_pred[0]
        precision, recall, thresholds = precision_recall_curve(self.val_y, y_pred)
        pr_auc = auc(recall, precision)
        roc_auc = roc_auc_score(self.val_y, y_pred)
        logs['custom_metric'] = pr_auc + roc_auc
        print ('val_aucs:', pr_auc, roc_auc)

In [19]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Embedding, Activation, Dropout, Softmax, Layer, InputSpec, Input, Dense, Lambda, TimeDistributed, Concatenate, Add
from tensorflow.keras import initializers, regularizers, constraints, Model
from tensorflow.python.keras.utils import tf_utils
from tensorflow.python.ops import array_ops
from tensorflow import nn

    
class CVE(Layer):
    def __init__(self, hid_units, output_dim):
        self.hid_units = hid_units
        self.output_dim = output_dim
        super(CVE, self).__init__()
        
    def build(self, input_shape): 
        self.W1 = self.add_weight(name='CVE_W1',
                            shape=(1, self.hid_units),
                            initializer='glorot_uniform',
                            trainable=True)
        self.b1 = self.add_weight(name='CVE_b1',
                            shape=(self.hid_units,),
                            initializer='zeros',
                            trainable=True)
        self.W2 = self.add_weight(name='CVE_W2',
                            shape=(self.hid_units, self.output_dim),
                            initializer='glorot_uniform',
                            trainable=True)
        super(CVE, self).build(input_shape)
        
    def call(self, x):
        x = K.expand_dims(x, axis=-1)
        x = K.dot(K.tanh(K.bias_add(K.dot(x, self.W1), self.b1)), self.W2)
        return x
        
    def compute_output_shape(self, input_shape):
        return input_shape + (self.output_dim,)
    
    
class Attention(Layer):
    
    def __init__(self, hid_dim):
        self.hid_dim = hid_dim
        super(Attention, self).__init__()

    def build(self, input_shape):
        d = input_shape.as_list()[-1]
        self.W = self.add_weight(shape=(d, self.hid_dim), name='Att_W',
                                 initializer='glorot_uniform',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.hid_dim,), name='Att_b',
                                 initializer='zeros',
                                 trainable=True)
        self.u = self.add_weight(shape=(self.hid_dim,1), name='Att_u',
                                 initializer='glorot_uniform',
                                 trainable=True)
        super(Attention, self).build(input_shape)
        
    def call(self, x, mask, mask_value=-1e30):
        attn_weights = K.dot(K.tanh(K.bias_add(K.dot(x,self.W), self.b)), self.u)
        mask = K.expand_dims(mask, axis=-1)
        attn_weights = mask*attn_weights + (1-mask)*mask_value
        attn_weights = K.softmax(attn_weights, axis=-2)
        return attn_weights
        
    def compute_output_shape(self, input_shape):
        return input_shape[:-1] + (1,)
    
    
class Transformer(Layer):
    
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0):
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = K.epsilon() * K.epsilon()
        super(Transformer, self).__init__()

    def build(self, input_shape):
        d = input_shape.as_list()[-1]
        if self.dk==None:
            self.dk = d//self.h
        if self.dv==None:
            self.dv = d//self.h
        if self.dff==None:
            self.dff = 2*d
        self.Wq = self.add_weight(shape=(self.N, self.h, d, self.dk), name='Wq',
                                 initializer='glorot_uniform', trainable=True)
        self.Wk = self.add_weight(shape=(self.N, self.h, d, self.dk), name='Wk',
                                 initializer='glorot_uniform', trainable=True)
        self.Wv = self.add_weight(shape=(self.N, self.h, d, self.dv), name='Wv',
                                 initializer='glorot_uniform', trainable=True)
        self.Wo = self.add_weight(shape=(self.N, self.dv*self.h, d), name='Wo',
                                 initializer='glorot_uniform', trainable=True)
        self.W1 = self.add_weight(shape=(self.N, d, self.dff), name='W1',
                                 initializer='glorot_uniform', trainable=True)
        self.b1 = self.add_weight(shape=(self.N, self.dff), name='b1',
                                 initializer='zeros', trainable=True)
        self.W2 = self.add_weight(shape=(self.N, self.dff, d), name='W2',
                                 initializer='glorot_uniform', trainable=True)
        self.b2 = self.add_weight(shape=(self.N, d), name='b2',
                                 initializer='zeros', trainable=True)
        self.gamma = self.add_weight(shape=(2*self.N,), name='gamma',
                                 initializer='ones', trainable=True)
        self.beta = self.add_weight(shape=(2*self.N,), name='beta',
                                 initializer='zeros', trainable=True)
        super(Transformer, self).build(input_shape)
        
    def call(self, x, mask, mask_value=-1e-30):
        mask = K.expand_dims(mask, axis=-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = K.dot(x, self.Wq[i,j,:,:])
                k = K.permute_dimensions(K.dot(x, self.Wk[i,j,:,:]), (0,2,1))
                v = K.dot(x, self.Wv[i,j,:,:])
                A = K.batch_dot(q,k)
                # Mask unobserved steps.
                A = mask*A + (1-mask)*mask_value
                # Mask for attention dropout.
                def dropped_A():
                    dp_mask = K.cast((K.random_uniform(shape=array_ops.shape(A))>=self.dropout), K.floatx())
                    return A*dp_mask + (1-dp_mask)*mask_value
                A = sc.smart_cond(K.learning_phase(), dropped_A, lambda: array_ops.identity(A))
                A = K.softmax(A, axis=-1)
                mha_ops.append(K.batch_dot(A,v))
            conc = K.concatenate(mha_ops, axis=-1)
            proj = K.dot(conc, self.Wo[i,:,:])
            # Dropout.
            proj = sc.smart_cond(K.learning_phase(), lambda: array_ops.identity(nn.dropout(proj, rate=self.dropout)),\
                                       lambda: array_ops.identity(proj))
            # Add & LN
            x = x+proj
            mean = K.mean(x, axis=-1, keepdims=True)
            variance = K.mean(K.square(x - mean), axis=-1, keepdims=True)
            std = K.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x*self.gamma[2*i] + self.beta[2*i]
            # FFN
            ffn_op = K.bias_add(K.dot(K.relu(K.bias_add(K.dot(x, self.W1[i,:,:]), self.b1[i,:])), 
                           self.W2[i,:,:]), self.b2[i,:,])
            # Dropout.
            ffn_op = sc.smart_cond(K.learning_phase(), lambda: array_ops.identity(nn.dropout(ffn_op, rate=self.dropout)),\
                                       lambda: array_ops.identity(ffn_op))
            # Add & LN
            x = x+ffn_op
            mean = K.mean(x, axis=-1, keepdims=True)
            variance = K.mean(K.square(x - mean), axis=-1, keepdims=True)
            std = K.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x*self.gamma[2*i+1] + self.beta[2*i+1]            
        return x
        
    def compute_output_shape(self, input_shape):
        return input_shape


def build_strats(D, max_len, V, d, N, he, dropout, forecast=False):
    demo = Input(shape=(D,))
    demo_enc = Dense(2*d, activation='tanh')(demo)
    demo_enc = Dense(d, activation='tanh')(demo_enc)
    varis = Input(shape=(max_len,))
    values = Input(shape=(max_len,))
    times = Input(shape=(max_len,))
    varis_emb = Embedding(V+1, d)(varis)
    cve_units = int(np.sqrt(d))
    values_emb = CVE(cve_units, d)(values)
    times_emb = CVE(cve_units, d)(times)
    comb_emb = Add()([varis_emb, values_emb, times_emb]) # b, L, d
#     demo_enc = Lambda(lambda x:K.expand_dims(x, axis=-2))(demo_enc) # b, 1, d
#     comb_emb = Concatenate(axis=-2)([demo_enc, comb_emb]) # b, L+1, d
    mask = Lambda(lambda x:K.clip(x,0,1))(varis) # b, L
#     mask = Lambda(lambda x:K.concatenate((K.ones_like(x)[:,0:1], x), axis=-1))(mask) # b, L+1
    cont_emb = Transformer(N, he, dk=None, dv=None, dff=None, dropout=dropout)(comb_emb, mask=mask)
    attn_weights = Attention(2*d)(cont_emb, mask=mask)
    fused_emb = Lambda(lambda x:K.sum(x[0]*x[1], axis=-2))([cont_emb, attn_weights])
    
    
    ## GloVe text embedding starts here
    max_length = 24135
    # embed text input
    texts = Input(shape=(max_length,))
    embedded_text = Embedding(vocab_size, 300, weights=[
    embedding_matrix], input_length=max_length, trainable=False)(texts)
    pooled = GlobalAveragePooling1D()(embedded_text)
    # flattened = Flatten()(embedded_text)
    
    text_0 = Dense(100, activation='relu')(pooled)
    
    
    # hidden_states = LSTM(64, return_sequences=True, name='lstm_layer')(embedded_text)
    # pooled = AveragePooling1D()(hidden_states)
    # text_0 = Dense(100, activation='relu')(hidden_states)
    
    # text_1 = Flatten()(hidden_states)
    
    # text_enc = Dense(22528, activation='relu')(texts)
    # text_enc = Dense(10000, activation='relu')(texts)
    # text_enc = Dense(5000, activation='relu')(texts)
    # text_enc = Dense(1000, activation='relu')(texts)
    
    # text_enc = Dense(d, activation='relu')(text_1)
    
    text_enc = Dense(d, activation='relu')(text_0)
    
    # text_enc = Dense(d * max_length, activation='relu')(text_0)
    # reshaped_text_enc = Reshape((max_length, d))(text_enc)
    
    # reshaped_text_enc = Reshape((d,))(text_enc)
    
    # conc = Concatenate(axis=-1)([fused_emb, text_enc, demo_enc])
    conc = Concatenate(axis=-1)([fused_emb, text_enc, demo_enc])
    fore_op = Dense(V)(conc)
    op = Dense(1, activation='sigmoid')(fore_op)
    model = Model([demo, times, values, varis, texts], op)
    if forecast:
        fore_model = Model([demo, times, values, varis, texts], fore_op)
        return [model, fore_model]
    return model

# To tune:
# 1. Transformer parameters. (N, h, dropout)
# 2. Normalization

## Pretrain on forecasting

In [20]:
# fore_train_ip[-1].shape

In [21]:
fore_valid_ip[-1].shape

(136823, 24135)

In [22]:
import gc
gc.collect()

0

In [23]:
# lr, batch_size, samples_per_epoch, patience = 0.0005, 8, 102400, 15
# # lr, batch_size, samples_per_epoch, patience = 0.0005, 1, 1024, 5
# d, N, he, dropout = 50, 2, 4, 0.2
# model, fore_model =  build_strats(D, fore_max_len, V, d, N, he, dropout, forecast=True)
# print (fore_model.summary())

# val_losses = []
# # best_val_loss = np.inf

# for i in list(range(100)):
#     gc.collect()
#     fore_path = 'Exp_GloVe/M/models/forecasting/forecasting_'+str(i+1)+'_epochs.h5'
#     fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
#     fore_model.load_weights(fore_path)

#     val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
#     val_losses.append(val_loss)
#     print(f'validation loss: {val_loss}')
#     gc.collect()
#     print(val_losses)
#     # if val_loss < best_val_loss:
#     #     best_val_loss = val_loss
#     #     best_epoch = i
#     # if (i-best_epoch)>patience:
#     #     print(f'best epoch: {best_epoch+1}')
#     #     break

In [24]:
# lr, batch_size, samples_per_epoch, patience = 0.0005, 8, 102400, 15
# # lr, batch_size, samples_per_epoch, patience = 0.0005, 1, 1024, 5
# d, N, he, dropout = 50, 2, 4, 0.2
# model, fore_model =  build_strats(D, fore_max_len, V, d, N, he, dropout, forecast=True)
# print (fore_model.summary())

# val_losses = [7.897054195404053, 7.160532474517822, 6.905936241149902, 6.643928527832031, 6.409193992614746, 6.341187953948975, 6.2095866203308105, 6.158576965332031, 6.146151065826416, 6.05009126663208, 6.095945358276367, 5.903674125671387, 5.900177001953125, 5.8338541984558105, 5.832334041595459, 6.0517897605896, 5.7025322914123535, 5.917352676391602, 5.697582721710205, 5.639064311981201, 5.710545063018799, 5.6525444984436035, 5.630661487579346, 5.600827693939209, 5.6292924880981445, 5.5955729484558105, 5.547638416290283, 5.512322902679443, 5.480807781219482, 5.4810285568237305, 5.625232696533203, 5.459251880645752]
# # best_val_loss = np.inf

# for i in range(32, 100):
#     gc.collect()
#     fore_path = 'Exp_GloVe/M/models/forecasting/forecasting_'+str(i+1)+'_epochs.h5'
#     fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
#     fore_model.load_weights(fore_path)

#     val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
#     val_losses.append(val_loss)
#     print(f'validation loss: {val_loss}')
#     gc.collect()
#     print(val_losses)
#     # if val_loss < best_val_loss:
#     #     best_val_loss = val_loss
#     #     best_epoch = i
#     # if (i-best_epoch)>patience:
#     #     print(f'best epoch: {best_epoch+1}')
#     #     break

In [25]:
# lr, batch_size, samples_per_epoch, patience = 0.0005, 8, 102400, 15
# # lr, batch_size, samples_per_epoch, patience = 0.0005, 1, 1024, 5
# d, N, he, dropout = 50, 2, 4, 0.2
# model, fore_model =  build_strats(D, fore_max_len, V, d, N, he, dropout, forecast=True)
# print (fore_model.summary())

# val_losses = [7.897054195404053, 7.160532474517822, 6.905936241149902, 6.643928527832031, 6.409193992614746, 6.341187953948975, 6.2095866203308105, 6.158576965332031, 6.146151065826416, 6.05009126663208, 6.095945358276367, 5.903674125671387, 5.900177001953125, 5.8338541984558105, 5.832334041595459, 6.0517897605896, 5.7025322914123535, 5.917352676391602, 5.697582721710205, 5.639064311981201, 5.710545063018799, 5.6525444984436035, 5.630661487579346, 5.600827693939209, 5.6292924880981445, 5.5955729484558105, 5.547638416290283, 5.512322902679443, 5.480807781219482, 5.4810285568237305, 5.625232696533203, 5.459251880645752, 5.541726589202881, 5.524600982666016, 5.501891136169434, 5.4433722496032715, 5.578892707824707]
# # best_val_loss = np.inf

# for i in range(37, 107):
#     gc.collect()
#     fore_path = 'Exp_GloVe/M/models/forecasting/forecasting_'+str(i+1)+'_epochs.h5'
#     fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
#     fore_model.load_weights(fore_path)

#     val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
#     val_losses.append(val_loss)
#     print(f'validation loss: {val_loss}')
#     gc.collect()
#     print(val_losses)
#     # if val_loss < best_val_loss:
#     #     best_val_loss = val_loss
#     #     best_epoch = i
#     # if (i-best_epoch)>patience:
#     #     print(f'best epoch: {best_epoch+1}')
#     #     break

In [26]:
# lr, batch_size, samples_per_epoch, patience = 0.0005, 8, 102400, 15
# # lr, batch_size, samples_per_epoch, patience = 0.0005, 1, 1024, 5
# d, N, he, dropout = 50, 2, 4, 0.2
# model, fore_model =  build_strats(D, fore_max_len, V, d, N, he, dropout, forecast=True)
# print (fore_model.summary())

# val_losses = [7.897054195404053, 7.160532474517822, 6.905936241149902, 6.643928527832031, 6.409193992614746, 6.341187953948975, 6.2095866203308105, 6.158576965332031, 6.146151065826416, 6.05009126663208, 6.095945358276367, 5.903674125671387, 5.900177001953125, 5.8338541984558105, 5.832334041595459, 6.0517897605896, 5.7025322914123535, 5.917352676391602, 5.697582721710205, 5.639064311981201, 5.710545063018799, 5.6525444984436035, 5.630661487579346, 5.600827693939209, 5.6292924880981445, 5.5955729484558105, 5.547638416290283, 5.512322902679443, 5.480807781219482, 5.4810285568237305, 5.625232696533203, 5.459251880645752, 5.541726589202881, 5.524600982666016, 5.501891136169434, 5.4433722496032715, 5.578892707824707, 5.409587383270264, 5.403265476226807, 5.390647888183594, 5.430344581604004, 5.38717794418335, 5.4096999168396, 5.494680881500244, 5.372311592102051, 5.484565258026123, 5.393592357635498, 5.4024434089660645, 5.410397529602051, 5.7557549476623535, 5.361731052398682, 5.3297200202941895, 5.334683895111084, 5.324537754058838, 5.394741058349609, 5.335954666137695, 5.439471244812012, 5.291817665100098, 5.447381496429443, 5.34607458114624, 5.378139495849609, 5.329391002655029, 5.346804141998291, 5.307310581207275, 5.30002498626709, 5.2998833656311035, 5.287087917327881, 5.30389404296875, 5.26768159866333, 5.317037582397461, 5.301440715789795, 5.3132429122924805, 5.353677749633789, 5.296224117279053, 5.268677711486816, 5.28335428237915, 5.296157360076904, 5.312287330627441, 5.2541823387146, 5.2621684074401855, 5.278446197509766, 5.239022254943848, 5.2674560546875, 5.226417541503906, 5.292505741119385, 5.2825140953063965, 5.303338050842285, 5.27216100692749, 5.24863862991333, 5.278128623962402, 5.233624458312988, 5.338470935821533, 5.209749698638916, 5.277923107147217, 5.258275985717773, 5.387875080108643, 5.241285800933838, 5.253940582275391, 5.23730993270874, 5.234818935394287]
# # best_val_loss = np.inf

# for i in range(100, 108):
#     gc.collect()
#     fore_path = 'Exp_GloVe/M/models/forecasting/forecasting_'+str(i+1)+'_epochs.h5'
#     fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
#     fore_model.load_weights(fore_path)

#     val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
#     val_losses.append(val_loss)
#     print(f'validation loss: {val_loss}')
#     gc.collect()
#     print(val_losses)
#     # if val_loss < best_val_loss:
#     #     best_val_loss = val_loss
#     #     best_epoch = i
#     # if (i-best_epoch)>patience:
#     #     print(f'best epoch: {best_epoch+1}')
#     #     break

In [27]:
lr, batch_size, samples_per_epoch, patience = 0.0005, 8, 102400, 15
# lr, batch_size, samples_per_epoch, patience = 0.0005, 1, 1024, 5
d, N, he, dropout = 50, 2, 4, 0.2
model, fore_model =  build_strats(D, fore_max_len, V, d, N, he, dropout, forecast=True)
print (fore_model.summary())

val_losses = [7.897054195404053, 7.160532474517822, 6.905936241149902, 6.643928527832031, 6.409193992614746, 6.341187953948975, 6.2095866203308105, 6.158576965332031, 6.146151065826416, 6.05009126663208, 6.095945358276367, 5.903674125671387, 5.900177001953125, 5.8338541984558105, 5.832334041595459, 6.0517897605896, 5.7025322914123535, 5.917352676391602, 5.697582721710205, 5.639064311981201, 5.710545063018799, 5.6525444984436035, 5.630661487579346, 5.600827693939209, 5.6292924880981445, 5.5955729484558105, 5.547638416290283, 5.512322902679443, 5.480807781219482, 5.4810285568237305, 5.625232696533203, 5.459251880645752, 5.541726589202881, 5.524600982666016, 5.501891136169434, 5.4433722496032715, 5.578892707824707, 5.409587383270264, 5.403265476226807, 5.390647888183594, 5.430344581604004, 5.38717794418335, 5.4096999168396, 5.494680881500244, 5.372311592102051, 5.484565258026123, 5.393592357635498, 5.4024434089660645, 5.410397529602051, 5.7557549476623535, 5.361731052398682, 5.3297200202941895, 5.334683895111084, 5.324537754058838, 5.394741058349609, 5.335954666137695, 5.439471244812012, 5.291817665100098, 5.447381496429443, 5.34607458114624, 5.378139495849609, 5.329391002655029, 5.346804141998291, 5.307310581207275, 5.30002498626709, 5.2998833656311035, 5.287087917327881, 5.30389404296875, 5.26768159866333, 5.317037582397461, 5.301440715789795, 5.3132429122924805, 5.353677749633789, 5.296224117279053, 5.268677711486816, 5.28335428237915, 5.296157360076904, 5.312287330627441, 5.2541823387146, 5.2621684074401855, 5.278446197509766, 5.239022254943848, 5.2674560546875, 5.226417541503906, 5.292505741119385, 5.2825140953063965, 5.303338050842285, 5.27216100692749, 5.24863862991333, 5.278128623962402, 5.233624458312988, 5.338470935821533, 5.209749698638916, 5.277923107147217, 5.258275985717773, 5.387875080108643, 5.241285800933838, 5.253940582275391, 5.23730993270874, 5.234818935394287, 5.2547078132629395, 5.271788597106934, 5.19520902633667, 5.232547283172607, 5.592817306518555, 5.308613300323486, 5.235623359680176, 5.170656204223633]

# [7.897054195404053, 7.160532474517822, 6.905936241149902, 6.643928527832031, 6.409193992614746, 6.341187953948975, 6.2095866203308105, 6.158576965332031, 6.146151065826416, 6.05009126663208, 6.095945358276367, 5.903674125671387, 5.900177001953125, 5.8338541984558105, 5.832334041595459, 6.0517897605896, 5.7025322914123535, 5.917352676391602, 5.697582721710205, 5.639064311981201, 5.710545063018799, 5.6525444984436035, 5.630661487579346, 5.600827693939209, 5.6292924880981445, 5.5955729484558105, 5.547638416290283, 5.512322902679443, 5.480807781219482, 5.4810285568237305, 5.625232696533203, 5.459251880645752, 5.541726589202881, 5.524600982666016, 5.501891136169434, 5.4433722496032715, 5.578892707824707, 5.409587383270264, 5.403265476226807, 5.390647888183594, 5.430344581604004, 5.38717794418335, 5.4096999168396, 5.494680881500244, 5.372311592102051, 5.484565258026123, 5.393592357635498, 5.4024434089660645, 5.410397529602051, 5.7557549476623535, 5.361731052398682, 5.3297200202941895, 5.334683895111084, 5.324537754058838, 5.394741058349609, 5.335954666137695, 5.439471244812012, 5.291817665100098, 5.447381496429443, 5.34607458114624, 5.378139495849609, 5.329391002655029, 5.346804141998291, 5.307310581207275, 5.30002498626709, 5.2998833656311035, 5.287087917327881, 5.30389404296875, 5.26768159866333, 5.317037582397461, 5.301440715789795, 5.3132429122924805, 5.353677749633789, 5.296224117279053, 5.268677711486816, 5.28335428237915, 5.296157360076904, 5.312287330627441, 5.2541823387146, 5.2621684074401855, 5.278446197509766, 5.239022254943848, 5.2674560546875, 5.226417541503906, 5.292505741119385, 5.2825140953063965, 5.303338050842285, 5.27216100692749, 5.24863862991333, 5.278128623962402, 5.233624458312988, 5.338470935821533, 5.209749698638916, 5.277923107147217, 5.258275985717773, 5.387875080108643, 5.241285800933838, 5.253940582275391, 5.23730993270874, 5.234818935394287]
# best_val_loss = np.inf

for i in range(108, 171):
    gc.collect()
    fore_path = 'Exp_GloVe/M/models/forecasting/forecasting_'+str(i+1)+'_epochs.h5'
    fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
    fore_model.load_weights(fore_path)

    val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
    val_losses.append(val_loss)
    print(f'validation loss: {val_loss}')
    gc.collect()
    print(val_losses)
    # if val_loss < best_val_loss:
    #     best_val_loss = val_loss
    #     best_epoch = i
    # if (i-best_epoch)>patience:
    #     print(f'best epoch: {best_epoch+1}')
    #     break

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 880)]        0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 880)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 880)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 880, 50)      6750        ['input_2[0][0]']                
                                                                                            


KeyboardInterrupt

